# SAM Demo - Complete Setup

**Snowcrest Asset Management (SAM) - Agentic AI for Asset Management**

This notebook creates:
- 24 CURATED tables with 14,000+ real securities
- 24 RAW document tables with documents from 69 templates
- 22 Cortex Search services
- 7 Cortex Analyst semantic views
- 8 Snowflake Intelligence agents

**Setup Time**: ~15-20 minutes

---

## Section 1: Environment Setup

In [ ]:
# Import Snowpark
from snowflake.snowpark.context import get_active_session
import sys
import os

# Get active session
session = get_active_session()

print(f"Current role: {session.get_current_role()}")
print(f"Current warehouse: {session.get_current_warehouse()}")
print(f"Current database: {session.get_current_database()}")

## Download Files from Git Repository

Download Python files and content templates from the Git stage.

In [ ]:
import os
import shutil

# Base stage path
GIT_STAGE = '@SAM_DEMO.PUBLIC.sam_demo_repo/branches/main'

# Create tmp directories
os.makedirs('/tmp/sam_demo/python', exist_ok=True)
os.makedirs('/tmp/sam_demo/content_library', exist_ok=True)

print("Downloading Python modules...")

# Download Python files
python_files = [
    'config.py', 'generate_structured.py', 'generate_unstructured.py',
    'build_ai.py', 'create_agents.py', 'create_semantic_views.py',
    'create_cortex_search.py', 'hydration_engine.py', 'extract_real_assets.py'
]

for f in python_files:
    try:
        session.file.get(f"{GIT_STAGE}/python/{f}", '/tmp/sam_demo/python/')
    except Exception as e:
        print(f"  Warning: {f} - {e}")

print(f"  Downloaded {len(python_files)} Python files")

In [ ]:
# Download content library templates
print("Downloading content library templates...")

# List all files in content_library
try:
    content_files = session.sql(f"LIST {GIT_STAGE}/content_library/").collect()
    
    # Download recursively
    downloaded = 0
    for row in content_files:
        file_path = row['name']
        if file_path.endswith('.md') or file_path.endswith('.yaml'):
            # Get relative path
            rel_path = file_path.split('/content_library/')[-1]
            local_dir = os.path.dirname(f'/tmp/sam_demo/content_library/{rel_path}')
            os.makedirs(local_dir, exist_ok=True)
            
            try:
                session.file.get(f"@{file_path}", local_dir + '/')
                downloaded += 1
            except:
                pass
    
    print(f"  Downloaded {downloaded} template files")
except Exception as e:
    print(f"  Note: Using embedded templates instead ({e})")

In [ ]:
# Add to Python path and set content library path
sys.path.insert(0, '/tmp/sam_demo/python')

# Import and update config
import config

# Override paths for Snowflake notebook environment
config.PROJECT_ROOT = '/tmp/sam_demo'
config.CONTENT_LIBRARY_PATH = '/tmp/sam_demo/content_library'

# Import other modules
import generate_structured
import generate_unstructured  
import build_ai
import create_agents

print("Modules imported successfully")
print(f"Database: {config.DATABASE['name']}")
print(f"Content library: {config.CONTENT_LIBRARY_PATH}")

## Section 2: Generate Structured Data

Create all dimension and fact tables.

**Time**: ~5-8 minutes

In [ ]:
# Generate structured data
print("Generating structured data model...")

scenarios = ['all']
generate_structured.build_all(session, scenarios, test_mode=False)

print("\nStructured data complete!")

## Section 3: Generate Documents

Generate documents from templates.

**Time**: ~3-5 minutes

In [ ]:
# Generate documents
print("Generating documents from templates...")

document_types = ['all']
generate_unstructured.build_all(session, document_types, test_mode=False)

print("\nDocument generation complete!")

## Section 4: Create AI Components

Create 22 Cortex Search services and 7 Cortex Analyst semantic views.

**Time**: ~3-5 minutes

In [ ]:
# Create AI components
print("Creating AI components...")

scenarios = ['all']
build_ai.build_all(session, scenarios, build_semantic=True, build_search=True, build_agents=False)

print("\nAI components created!")

## Section 5: Create Agents

Create all 8 AI agents.

**Time**: ~2-3 minutes

In [ ]:
# Create agents
print("Creating Snowflake Intelligence agents...")

scenarios = ['all']
created, failed = create_agents.create_all_agents(session, scenarios)

print(f"\nAgents created: {created}, Failed: {failed}")

## Section 6: Verification

In [ ]:
# Verify setup
print("Verifying setup...\n")

curated = session.sql("SHOW TABLES IN SAM_DEMO.CURATED").collect()
print(f"CURATED tables: {len(curated)} (expected: 24)")

raw = session.sql("SHOW TABLES IN SAM_DEMO.RAW").collect()
print(f"RAW tables: {len(raw)} (expected: 24)")

search = session.sql("SHOW CORTEX SEARCH SERVICES IN SAM_DEMO.AI").collect()
print(f"Cortex Search services: {len(search)} (expected: 22)")

semantic = session.sql("SHOW SEMANTIC VIEWS IN SAM_DEMO.AI").collect()
print(f"Semantic views: {len(semantic)} (expected: 7)")

try:
    agents = session.sql("SHOW AGENTS IN SAM_DEMO.AI").collect()
    print(f"Agents: {len(agents)} (expected: 8)")
except:
    print("Agents: (run SHOW AGENTS manually)")

print("\n" + "="*50)
print("SAM DEMO SETUP COMPLETE!")
print("="*50)
print("\nNext Steps:")
print("1. Open Snowflake Intelligence UI")
print("2. Select an agent (e.g., Portfolio Copilot)")
print("3. Try: 'What are my top 10 holdings?'")